In [1]:
## General Imports
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Data wrangling and cleaning

Create a wrangling function to automate dataset cleaning and manipulation based on what observed in the EDA.

In [2]:
def wrangling(filepath_values, filepath_labels):
    #--- import the data ---#
    vals = pd.read_csv(filepath_values)
    labels = pd.read_csv(filepath_labels)

    df = pd.merge(
        left=vals,
        right=labels,
        on='id'
    )

    #--- convert to datetime and add month_recorded column ---#
    df['date_recorded'] = pd.to_datetime(df.date_recorded, format='%Y-%m-%d')
    df['month_recorded'] = df.date_recorded.dt.month

    #--- fix the population field whereas 0 is actually a null value ---#
    df.population.replace(0.0,np.nan,inplace=True)
    df['population'].fillna(df.groupby(['region', 'district_code'])['population'].transform('median'), inplace=True)
    df['population'].fillna(df.groupby(['region'])['population'].transform('median'), inplace=True)
    df['population'].fillna(df['population'].median(), inplace=True)

    #--- fix the gps_height field whereas 0 is actually a null value ---#
    df.gps_height.replace(0.0,np.nan,inplace=True)
    df['gps_height'].fillna(df.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
    df['gps_height'].fillna(df.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
    df['gps_height'].fillna(df['gps_height'].mean(), inplace=True)

    #--- fix the construction_year field whereas 0 is actually a null value ---#
    # df.loc[(df.date_recorded.dt.year - df.construction_year) < 0, 'construction_year'] = 0.0
    df['construction_year'].replace(0.0, np.nan, inplace=True)
    df['construction_year'].fillna(df.groupby(['region', 'district_code'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df.groupby(['region'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df['construction_year'].median(), inplace=True)

    #--- short rains are usually between Oct and Dec, long rains between Mar and May ---#
    rains = [3,4,5,10,11,12]

    def rainy_season(series):
        if series in rains:
            return 1
        else:
            return 0

    df['rainy_season'] = df.month_recorded.apply(rainy_season)

    #--- adding column age of water point at the time of assessment ---#
    #--- if the age is negative (mistake in recorded date or construction year), round it to 0 ---#
    df['age_yrs'] = df.date_recorded.dt.year - df.construction_year
    df.loc[df.age_yrs < 0, 'age_yrs'] = 0

    #--- changing column 'num_private' to boolean and renaming it to 'private_conn' ---#
    df.loc[df.num_private == 0, 'num_private'] = 0
    df.loc[df.num_private > 0, 'num_private'] = 1
    df.rename(columns={'num_private': 'private_conn'}, inplace=True)
    
    #--- in column 'installer' replace with 'other' anything which is not DWE or government ---#
    df['installer'] = df.installer.fillna('unknown')

    dwe_pattern = []
    gov_pattern = []

    for i in df.installer.unique():
        import re
        if re.search("(dwe)", i, re.IGNORECASE):
            dwe_pattern.append(i)
        elif re.search("(government)", i, re.IGNORECASE):
            gov_pattern.append(i)

    def installer_group(data):
        if data.installer in dwe_pattern:
            return 'DWE'
        elif data.installer in gov_pattern:
            return 'government'
        else:
            return 'other'

    df['installer'] = df.apply(installer_group, axis=1)

    #--- drop unnecessary columns ---#
    cols_to_drop = [
        'amount_tsh', 
        'funder', 
        'longitude', 'latitude', 
        'wpt_name', 'subvillage', 'region_code', 'district_code', 'lga', 'ward', 'scheme_name',  
        'recorded_by',
        'scheme_management', 'management',
        'extraction_type', 'extraction_type_group', 
        'payment',
        'water_quality', 'quantity', 'source', 'source_class', 'waterpoint_type',
        'construction_year', 'date_recorded' 
    ]
    
    df.drop(cols_to_drop, axis=1, inplace=True)

    #--- the remaining columns with null values are 'public_meeting' and 'permit' ---#
    #--- replace null value with 'unknown' ---#
    # df.replace(True, 1, inplace=True)
    # df.replace(False, 0, inplace=True)
    df.fillna('unknown', inplace=True)

    return df

In [7]:
def wrangling_2(filepath_values, filepath_labels):

    #--- import the data ---#
    vals = pd.read_csv(filepath_values)
    labels = pd.read_csv(filepath_labels)

    df = pd.merge(
        left=vals,
        right=labels,
        on='id'
    )
    #--- convert to datetime and add month_recorded column ---#
    df['date_recorded'] = pd.to_datetime(df.date_recorded, format='%Y-%m-%d')
    df['month_recorded'] = df.date_recorded.dt.month

    #--- fix the population field whereas 0 is actually a null value ---#
    df.population.replace(0.0,np.nan,inplace=True)
    df['population'].fillna(df.groupby(['region', 'district_code'])['population'].transform('median'), inplace=True)
    df['population'].fillna(df.groupby(['region'])['population'].transform('median'), inplace=True)
    df['population'].fillna(df['population'].median(), inplace=True)

    #--- fix the gps_height field whereas 0 is actually a null value ---#
    df.gps_height.replace(0.0,np.nan,inplace=True)
    df['gps_height'].fillna(df.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
    df['gps_height'].fillna(df.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
    df['gps_height'].fillna(df['gps_height'].mean(), inplace=True)

    #--- fix the latitude and longitude fields whereas 0 is actually a null value ---#
    df.latitude.replace(0.0,np.nan,inplace=True)
    df.longitude.replace(0.0,np.nan,inplace=True)
    df['latitude'].fillna(df.groupby(['region', 'district_code'])['latitude'].transform('mean'), inplace=True)
    df['latitude'].fillna(df.groupby(['region'])['latitude'].transform('mean'), inplace=True)
    df['latitude'].fillna(df['latitude'].mean(), inplace=True)
    df['longitude'].fillna(df.groupby(['region', 'district_code'])['longitude'].transform('mean'), inplace=True)
    df['longitude'].fillna(df.groupby(['region'])['longitude'].transform('mean'), inplace=True)
    df['longitude'].fillna(df['longitude'].mean(), inplace=True)

    #--- fix the construction_year field whereas 0 is actually a null value ---#
    # df.loc[(df.date_recorded.dt.year - df.construction_year) < 0, 'construction_year'] = 0.0
    df['construction_year'].replace(0.0, np.nan, inplace=True)
    df['construction_year'].fillna(df.groupby(['region', 'district_code'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df.groupby(['region'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    df['construction_year'].fillna(df['construction_year'].median(), inplace=True)

    #--- short rains are usually between Oct and Dec, long rains between Mar and May ---#
    rains = [3,4,5,10,11,12]

    def rainy_season(series):
        if series in rains:
            return 1
        else:
            return 0

    df['rainy_season'] = df.month_recorded.apply(rainy_season)

    #--- adding column age of water point at the time of assessment ---#
    #--- if the age is negative (mistake in recorded date or construction year), round it to 0 ---#
    df['age_yrs'] = df.date_recorded.dt.year - df.construction_year
    df.loc[df.age_yrs < 0, 'age_yrs'] = 0

    #--- changing column 'num_private' to boolean and renaming it to 'private_conn' ---#
    df.loc[df.num_private == 0, 'num_private'] = 0
    df.loc[df.num_private > 0, 'num_private'] = 1
    df.rename(columns={'num_private': 'private_conn'}, inplace=True)
    
    #--- in column 'installer' replace with 'other' anything which is not DWE or government ---#
    df['installer'] = df.installer.str.lower()
    df['installer'] = df.installer.fillna('unknown')

    dwe_pattern = []
    gov_pattern = []

    for i in df.installer.unique():
        import re
        if re.search("(dwe)", i, re.IGNORECASE):
            dwe_pattern.append(i)
        elif re.search("(government)", i, re.IGNORECASE):
            gov_pattern.append(i)

    def installer_group(data):
        if data.installer in dwe_pattern:
            return 'DWE'
        elif data.installer in gov_pattern:
            return 'government'
        else:
            return 'other'

    df['installer'] = df.apply(installer_group, axis=1)

    #--- in column 'funder' replace with 'other' all missing values ---#
    #--- and then factorize it (there are too many distinct value to create dummy) ---#
    df['funder'] = df.funder.str.lower()
    df['funder'] = df.funder.fillna('other')
    df['funder'] = pd.factorize(df.funder)[0]

    #--- drop unnecessary columns ---#
    cols_to_drop = [
        'amount_tsh', 
        # 'funder', 
        'wpt_name', 'subvillage', 'region_code', 'district_code', 'lga', 'ward', 'scheme_name',  
        'recorded_by',
        'scheme_management', 'management',
        'extraction_type', 'extraction_type_group', 
        'payment',
        'water_quality', 'quantity', 'source', 'source_class', 'waterpoint_type',
        'construction_year', 'date_recorded',
    ]
    
    df.drop(cols_to_drop, axis=1, inplace=True)

    #--- the remaining columns with null values are 'public_meeting' and 'permit' ---#
    #--- replace null value with 'unknown' ---#
    # df.replace(True, 1, inplace=True)
    # df.replace(False, 0, inplace=True)
    df.fillna('unknown', inplace=True)
    
    #--- factorize the target variable ---#
    df.replace('functional', 0, inplace=True)
    df.replace('non functional', 1, inplace=True)
    df.replace('functional needs repair', 2, inplace=True)

    return df

In [8]:
filepath_values = "~/Coding/drivendata/pump-it-up/data/raw/TrainingSetValues.csv"
filepath_labels = "~/Coding/drivendata/pump-it-up/data/raw/TrainingSetLabels.csv"

# df = wrangling(filepath_values, filepath_labels)
df = wrangling_2(filepath_values, filepath_labels)
df.sample(10)

,id,funder,gps_height,installer,longitude,latitude,private_conn,basin,region,population,...,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type_group,status_group,month_recorded,rainy_season,age_yrs
40592,42511,23,8.000000,other,38.988097,-6.538720,0,Wami / Ruvu,Pwani,40.0,...,commercial,per bucket,good,enough,river/lake,communal standpipe,0,3,1,1.0
2861,47067,68,427.000000,other,37.573338,-10.856133,0,Ruvuma / Southern Coast,Ruvuma,1.0,...,user-group,unknown,good,enough,borehole,other,1,2,0,1.0
11859,52897,490,1764.000000,DWE,36.534056,-3.288332,0,Internal,Arusha,100.0,...,user-group,never pay,good,enough,spring,communal standpipe,0,5,1,3.0
32782,21644,42,1450.000000,other,35.311283,-4.622598,0,Internal,Manyara,256.0,...,user-group,per bucket,salty,dry,spring,communal standpipe,1,2,0,8.0
24171,64802,11,1057.545585,government,33.862999,-9.612584,0,Lake Nyasa,Mbeya,200.0,...,user-group,never pay,good,dry,spring,communal standpipe,1,7,0,9.0
32888,44552,395,1205.511166,other,32.638599,-2.816879,0,Lake Victoria,Mwanza,450.0,...,user-group,never pay,salty,enough,borehole,hand pump,0,7,0,16.0
39537,37832,1534,1453.000000,other,37.616677,-3.051659,0,Internal,Kilimanjaro,1.0,...,parastatal,never pay,good,insufficient,rainwater harvesting,communal standpipe,0,2,0,3.0
6288,49933,25,1427.000000,other,34.501954,-4.825662,0,Internal,Singida,1.0,...,user-group,unknown,unknown,dry,shallow well,other,1,1,0,33.0
36437,60315,63,1388.000000,other,34.460181,-4.803516,0,Internal,Singida,225.0,...,user-group,on failure,good,insufficient,borehole,hand pump,1,2,0,5.0
5660,20157,210,2240.000000,other,34.436609,-9.303841,0,Lake Nyasa,Iringa,100.0,...,user-group,on failure,good,enough,spring,communal standpipe,0,11,1,15.0


In [12]:
report = ProfileReport(df, title='PumpItUp_EDA')
report.to_file("clean_data_EDA.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
display(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   funder                 59400 non-null  int64  
 2   gps_height             59400 non-null  float64
 3   installer              59400 non-null  object 
 4   longitude              59400 non-null  float64
 5   latitude               59400 non-null  float64
 6   private_conn           59400 non-null  int64  
 7   basin                  59400 non-null  object 
 8   region                 59400 non-null  object 
 9   population             59400 non-null  float64
 10  public_meeting         59400 non-null  object 
 11  permit                 59400 non-null  object 
 12  extraction_type_class  59400 non-null  object 
 13  management_group       59400 non-null  object 
 14  payment_type           59400 non-null  object 
 15  qu

None

,id,funder,gps_height,longitude,latitude,private_conn,population,status_group,month_recorded,rainy_season,age_yrs
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,132.857576,1057.545585,35.106608,-5.706033e+00,0.012744,288.198695,0.529596,4.375640,0.499630,13.633889
std,21453.128371,244.975036,507.785910,2.580356,2.946019e+00,0.112169,461.673811,0.628080,3.029247,0.500004,10.560510
min,0.000000,0.000000,-90.000000,29.607122,-1.164944e+01,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,18519.750000,20.000000,838.000000,33.271875,-8.540621e+00,0.000000,80.000000,0.000000,2.000000,0.000000,7.000000
50%,37061.500000,40.000000,1057.545585,34.908743,-5.021597e+00,0.000000,200.000000,0.000000,3.000000,0.000000,11.000000
75%,55656.500000,129.000000,1350.981707,37.178387,-3.326156e+00,0.000000,410.000000,1.000000,7.000000,1.000000,17.000000
max,74247.000000,1896.000000,2770.000000,40.345193,-2.000000e-08,1.000000,30500.000000,2.000000,12.000000,1.000000,53.000000


In [17]:
#--- Saving the clean dataframe to a csv file for future steps ---#

df.to_csv("~/Coding/drivendata/pump-it-up/data/df_clean.csv", index=False)